In [20]:
from elasticsearch import Elasticsearch

In [21]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [22]:
if es.indices.exists('es0'):
    es.indices.delete('es0')
es.indices.create('es0', body={
    "mappings": {
        "properties": {
            "my_nested_field": {
                "type": "nested",
                "properties": {
                    "my_text_field": {
                        "type": "text",
                        "analyzer": "ik_max_word"
                    }
                }
            }
        }
    }
})

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'es0'}

In [23]:
'''
若my_nested_field字段为数组类型,即
    * "my_nested_field": ["轻度脂肪肝", "脂肪肝 甲状腺结节 双眼屈光不正"]
    * "my_nested_field":["脂肪肝]
    * "my_nested_field":["轻度脂肪肝"]
    * ......
则计算的score为query与数组所有元素组合的BM25分数

解析:若query=a,s=['A', 'B', 'C'] ======> score=BM25(a, 'A' + 'B' + 'C');注意:数组元素之间的position_increment_gap默认为100,数组不会跨元素分词

解析:若query=a,s=[{"q": "A"}, {"q": "B"}, {"q": "C"}]
1. "score_mode"="min" ======> score=min(BM25(a, A), BM25(a, B), BM25(a, C))
2. "score_mode"="max" ======> score=max(BM25(a, A), BM25(a, B), BM25(a, C))
3. "score_mode"="avg" ======> score=sum(BM25(a, A), BM25(a, B), BM25(a, C)) / len(s)
4. "score_mode"="sum" ======> score=sum(BM25(a, A), BM25(a, B), BM25(a, C))
'''
es.index(index='es0', id=0, body={
    "my_nested_field": [
        {
            "my_text_field": "轻度脂肪肝"
        },
        {
            "my_text_field": "脂肪肝 甲状腺结节 双眼屈光不正"
        }
    ]
})
es.index(index='es0', id=1, body={
    "my_nested_field": [
        {
            "my_text_field": "脂肪肝"
        }
    ]
})
es.index(index='es0', id=2, body={
    "my_nested_field": [
        {
            "my_text_field": "轻度脂肪肝"
        }
    ]
})
es.index(index='es0', id=3, body={
    "my_nested_field": [
        {
            "my_text_field": "轻－中度脂肪肝"
        }
    ]
})
es.index(index='es0', id=5, body={
    "my_nested_field": [
        {
            "my_text_field": "脂肪肝 甲状腺结节 双眼屈光不正"
        }
    ]
})

es.index(index='es0', id=5, body={
    "my_nested_field": [
        {
            "my_text_field": "脂肪肝 甲状腺结节 双眼屈光不正"
        }
    ]
})
es.index(index='es0', id=6, body={
    "my_nested_field": [
        {
            "my_text_field": "非均匀性脂肪肝"
        }
    ]
})

{'_index': 'es0',
 '_type': '_doc',
 '_id': '6',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 6,
 '_primary_term': 1}

In [24]:
es.search(index='es0', body={
    "query": {
        "nested": {
            "path": "my_nested_field",
            "query": {
                "match": {
                    "my_nested_field.my_text_field": "脂肪肝"
                }
            },
            # min: Uses the lowest relevance score of all matching child objects.
            "score_mode": "min"
        }
    }
})

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [25]:
es.search(index='es0', body={
    "query": {
        "nested": {
            "path": "my_nested_field",
            "query": {
                "match": {
                    "my_nested_field.my_text_field": "脂肪肝"
                }
            },
            # max: Uses the highest relevance score of all matching child objects.
            "score_mode": "max"
        }
    }
})

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 6, 'relation': 'eq'},
  'max_score': 0.2167061,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '1',
    '_score': 0.2167061,
    '_source': {'my_nested_field': [{'my_text_field': '脂肪肝'}]}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '0',
    '_score': 0.19983816,
    '_source': {'my_nested_field': [{'my_text_field': '轻度脂肪肝'},
      {'my_text_field': '脂肪肝 甲状腺结节 双眼屈光不正'}]}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '2',
    '_score': 0.19983816,
    '_source': {'my_nested_field': [{'my_text_field': '轻度脂肪肝'}]}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '3',
    '_score': 0.1854065,
    '_source': {'my_nested_field': [{'my_text_field': '轻－中度脂肪肝'}]}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '6',
    '_score': 0.17291892,
    '_source': {'my_nested_field': [{'my_text_field': '非均匀性脂肪肝'}]}},
   {'_index'

In [26]:
es.search(index='es0', body={
    "query": {
        "nested": {
            "path": "my_nested_field",
            "query": {
                "match": {
                    "my_nested_field.my_text_field": "脂肪肝"
                }
            },
            # max: Add together the relevance scores of all matching child objects.
            "score_mode": "sum"
        }
    }
})

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 6, 'relation': 'eq'},
  'max_score': 0.34369054,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '0',
    '_score': 0.34369054,
    '_source': {'my_nested_field': [{'my_text_field': '轻度脂肪肝'},
      {'my_text_field': '脂肪肝 甲状腺结节 双眼屈光不正'}]}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '1',
    '_score': 0.2167061,
    '_source': {'my_nested_field': [{'my_text_field': '脂肪肝'}]}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '2',
    '_score': 0.19983816,
    '_source': {'my_nested_field': [{'my_text_field': '轻度脂肪肝'}]}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '3',
    '_score': 0.1854065,
    '_source': {'my_nested_field': [{'my_text_field': '轻－中度脂肪肝'}]}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '6',
    '_score': 0.17291892,
    '_source': {'my_nested_field': [{'my_text_field': '非均匀性脂肪肝'}]}},
   {'_index

In [27]:
es.search(index='es0', body={
    "query": {
        "nested": {
            "path": "my_nested_field",
            "query": {
                "match": {
                    "my_nested_field.my_text_field": "脂肪肝"
                }
            },
            # avg (Default): Use the mean relevance score of all matching child objects.
            "score_mode": "avg"
        }
    }
})

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 6, 'relation': 'eq'},
  'max_score': 0.2167061,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '1',
    '_score': 0.2167061,
    '_source': {'my_nested_field': [{'my_text_field': '脂肪肝'}]}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '2',
    '_score': 0.19983816,
    '_source': {'my_nested_field': [{'my_text_field': '轻度脂肪肝'}]}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '3',
    '_score': 0.1854065,
    '_source': {'my_nested_field': [{'my_text_field': '轻－中度脂肪肝'}]}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '6',
    '_score': 0.17291892,
    '_source': {'my_nested_field': [{'my_text_field': '非均匀性脂肪肝'}]}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '0',
    '_score': 0.17184527,
    '_source': {'my_nested_field': [{'my_text_field': '轻度脂肪肝'},
      {'my_text_field': '脂肪肝 甲状腺结节 双眼屈光不正'}]}},
   {'_index'